In [ ]:
from DeepER import init_embeddings_index, init_embeddings_model, init_DeepER_model, train_model_ER, replace_last_layer, model_statistics
from experimental_similarity import mono_vector, cosine_similarity_vector, distance_similarity_vector
from csv2dataset import splitting_dataSet, csv_2_datasetALTERNATE, csvTable2datasetRANDOM,parsing_anhai_data
from generate_similarity_vector import generate_similarity_vector
from data_reg import sim_hamming
from keras.models import load_model
from random import shuffle
import utilist as uls

In [ ]:
# Caricamento strutture dati e modelli ausiliari.
embeddings_index = init_embeddings_index('embeddings/glove.6B.100d.txt')
emb_dim = len(embeddings_index['cat']) # :3
embeddings_model, tokenizer = init_embeddings_model(embeddings_index)

In [ ]:
# Caricamento dati e split iniziale.
if LOAD_FROM_DISK_DATASET:
    
    # Carica dataset salvato su disco.
    data = uls.load_list(f'dataset_{DATASET_NAME}')
    match_number=sum(map(lambda x : x[3] == 1, data))
    print("match_number: " + str(match_number))
    print("len all dataset: "+ str(len(data)))

else:
    
    GROUND_TRUTH_FILE = 'matches_fodors_zagats.csv'# Esempio: 'matches_walmart_amazon.csv'
    # Necessario inserire le tabelle nell'ordine corrispondente alle coppie della ground truth.
    TABLE1_FILE = 'fodors.csv'# Esempio: 'walmart.csv'
    TABLE2_FILE = 'zagats.csv'# Esempio: 'amazon.csv'

    # Coppie di attributi considerati allineati.
    att_indexes = [(1, 1), (2, 2), (3, 3), (4, 4),(5, 5), (6, 6)]# Esempio: [(5, 9), (4, 5), (3, 3), (14, 4), (6, 11)]


    # Crea il dataset.
    data = csv_2_datasetALTERNATE(GROUND_TRUTH_FILE, TABLE1_FILE, TABLE2_FILE, att_indexes)
    #per i dataset di Anhai
    #data=parsing_anhai_data(GROUND_TRUTH_FILE, TABLE1_FILE, TABLE2_FILE, att_indexes)
    
    # Salva dataset su disco.
    uls.save_list(data, f'dataset_{DATASET_NAME}')

    
# Dataset per DeepER classico: [(tupla1, tupla2, label), ...].
deeper_data = list(map(lambda q: (q[0], q[1], q[3]), data))


# Taglia attributi se troppo lunghi
# Alcuni dataset hanno attributi con descrizioni molto lunghe.
# Questo filtro limita il numero di caratteri di un attributo a 1000.
def shrink_data(data):
    
    def cut_string(s):
        if len(s) >= 1000:
            return s[:1000]
        else:
            return s
    
    temp = []
    for t1, t2, lb in data:
        t1 = list(map(cut_string, t1))
        t2 = list(map(cut_string, t2))
        temp.append((t1, t2, lb))
        
    return temp

deeper_data = shrink_data(deeper_data)


# Split in training set e test set.
def split_training_test(data, SPLIT_FACTOR = 0.8):     
    bound = int(len(data) * SPLIT_FACTOR)
    train = data[:bound]
    test = data[bound:]
    
    return train, test


# Tutti i successivi addestramenti partiranno dal 100% di deeper_train (80% di tutti i dati).
# Le tuple in deeper_test non verranno mai usate per addestrare ma solo per testare i modelli.
deeper_train, deeper_test = split_training_test(deeper_data)

In [ ]:
from DeepER import data2Inputs
table1, table2, labels = data2Inputs(deeper_train, tokenizer, categorical=False)
embeddings = embeddings_model.predict([table1,table2])

In [ ]:
test_embeddings = [embeddings[0][0:8],embeddings[1][0:8]]
from keras.models import Model
layer_name = 'Similarity'
intermediate_layer_model = Model(inputs=deeper_model_100.input,
                                 outputs=deeper_model_100.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(test_embeddings)